In [19]:
import pdfplumber
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
import re
import string
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [20]:
def vectorizeText():
    
    # get counts and remove stop words
#     cv = CountVectorizer(stop_words='english')
#     X = cv.fit_transform(page_sentences)
#     df_cv = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())  
#     df_cv.info()
#     print(df_cv.columns)
#     print(df_cv)

    # get term frequencies and remove stop words
    cv_tfidf = TfidfVectorizer(stop_words='english')
    X_tfidf = cv_tfidf.fit_transform(page_sentences)
    df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=cv_tfidf.get_feature_names())
    df_tfidf.info()
    print(df_tfidf.columns)
    print(df_tfidf)      


def cleanText(page):
    page_text = this_page.extract_text()
    
    clean_text = page_text
    
    # remove punctuation
    clean_text = re.sub('[%s]' % re.escape(string.punctuation), ' ', clean_text)
    # make lower case
    clean_text = clean_text.lower()
    
    # remove numbers - maybe not necessary
#     clean_text = re.sub('\w*\d\w*', ' ', clean_text)
      
    # tokenize the text either in individual words or sentences
    clean_text = word_tokenize(clean_text)
#     page_sentences = sent_tokenize(clean_text) # this doesn't work if punctuation has already been removed, can't clean the punctuation with the same line of code as above once it's been tokenized

    # remove punctuation which has been reintroduced after tokenizing
    clean_text = [word for word in clean_text if word.isalpha()]

    # remove stop words (can do this manually or automatically with the vectorizer)
    clean_text = [word for word in clean_text if word not in stop_words]
    
    # stem words
    clean_text = [porter.stem(word) for word in clean_text]

    
    print(clean_text)
#     print(page_sentences)
    
#     print('Num words on page = ', len(page_words))
#     print('Num sentences on page = ', len(page_sentences))

        
  


    
#     twograms = list(ngrams(page_words,2))
#     print(twograms)
    

    

In [21]:
with pdfplumber.open("../StormlightArchiveBooks/The_Way_of_Kings_The_Stormlight_Archive_Book_1.pdf") as pdf:
    this_page = pdf.pages[24]
#     print(this_page.extract_text())
    cleanText(this_page)
    


['parshendi', 'agre', 'treati', 'send', 'assassin', 'night', 'sign', 'gemston', 'glow', 'wall', 'hallway', 'king', 'gavilar', 'like', 'lavish', 'display', 'know', 'leav', 'sourc', 'power', 'szeth', 'use', 'lash', 'thing', 'szeth', 'seen', 'millennia', 'histori', 'time', 'nonexist', 'legend', 'horribl', 'inaccur', 'szeth', 'peek', 'back', 'corridor', 'one', 'guard', 'intersect', 'saw', 'point', 'yell', 'szeth', 'made', 'sure', 'got', 'good', 'look', 'duck', 'away', 'took', 'deep', 'breath', 'ran', 'draw', 'stormlight', 'lantern', 'bodi', 'came', 'aliv', 'speed', 'increas', 'muscl', 'burst', 'energi', 'light', 'becam', 'storm', 'insid', 'blood', 'thunder', 'ear', 'terribl', 'wonder', 'time', 'two', 'corridor', 'one', 'side', 'threw', 'open', 'door', 'storag', 'room', 'hesit', 'long', 'enough', 'guard', 'round', 'corner', 'see', 'dash', 'room', 'prepar', 'full', 'lash', 'rais', 'arm', 'command', 'stormlight', 'pool', 'caus', 'skin', 'burst', 'alight', 'radianc', 'hand', 'toward', 'doorfra